In [3]:
# Name: Kathiriya Ranjit (R00183586)
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from textblob import Word
import re
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ranjitsmac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


    1.    Load Dataset:

    a.    Load dataset:
        i.    In subtaskA_data_all.csv, contain 10,000 entries with column name id,falsesent,optionA, optionB, and OptionC.
        ii.    In subtaskA_answers_all.csv, contain 10,000 entries with column name id and sent0 of the sentence, which the option is true either in A,B,C.
    

In [4]:
df_all = pd.read_csv('./TrainingData/subtaskB_data_all.csv')

In [5]:
df_all.head()

,id,FalseSent,OptionA,OptionB,OptionC
0,0,He poured orange juice on his cereal.,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...
1,1,He drinks apple.,Apple juice are very tasty and milk too,Apple can not be drunk,Apple cannot eat a human
2,2,"Jeff ran 100,000 miles today","100,000 miles is way to long for one person to...","Jeff is a four letter name and 100,000 has six...","100,000 miles is longer than 100,000 km."
3,3,I sting a mosquito,A human is a mammal,A human is omnivorous,A human has not stings
4,4,A giraffe is a person.,Giraffes can drink water from a lake.,A giraffe is not a human being.,.Giraffes usually eat leaves.


In [6]:
df_lables = pd.read_csv('./TrainingData/subtaskB_answers_all.csv',header='infer',names=['id','sen0_lbl'])

In [7]:
df_lables.head()

,id,sen0_lbl
0,0,B
1,1,B
2,2,A
3,3,C
4,4,B


In [49]:
# Merge dataset withrespect to id.
df = pd.merge(df_all,df_lables, on='id')

In [50]:
# Drop falsesent
df.drop('FalseSent',axis=1,inplace=True)

In [51]:
df.head(1)

,id,OptionA,OptionB,OptionC,sen0_lbl
0,0,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...,B


In [52]:
# provide numbering for lables for eg. A=1,B=2, and C=3
df['sen0_lbl'] = df["sen0_lbl"].apply(lambda x: 1 if x=='A' else (2 if x=='B' else 3))

In [53]:
df.head(1)

,id,OptionA,OptionB,OptionC,sen0_lbl
0,0,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...,2


In [54]:
# Create 3 columns names lblA,lblB, and lblC and asssign 0 in all. 
df['lblA'] = 0
df['lblB'] = 0
df['lblC'] = 0

In [57]:
# Assign values into lblA,lblB, and lblC from sen0_lbl, if it is true then 1 or else 0
df['lblA'] = df["sen0_lbl"].apply(lambda x: 1 if x== 1 else 0)
df['lblB'] = df["sen0_lbl"].apply(lambda x: 1 if x== 2 else 0)
df['lblC'] = df["sen0_lbl"].apply(lambda x: 1 if x== 3 else 0)

In [59]:
# drop sen0_lbl because we have sepreated this data into lblA,lblB, and lblC
df.drop('sen0_lbl',axis=1,inplace=True)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 7 columns):
id         10000 non-null int64
OptionA    10000 non-null object
OptionB    10000 non-null object
OptionC    9997 non-null object
lblA       10000 non-null int64
lblB       10000 non-null int64
lblC       10000 non-null int64
dtypes: int64(4), object(3)
memory usage: 625.0+ KB


In [63]:
# as we found that null value is present in optionC, so removing that perticular rows from dataset.
df['OptionC'].replace('', np.nan, inplace=True)

df.dropna(subset=['OptionC'],inplace=True)

In [65]:
df.head()

,id,OptionA,OptionB,OptionC,lblA,lblB,lblC
0,0,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...,0,1,0
1,1,Apple juice are very tasty and milk too,Apple can not be drunk,Apple cannot eat a human,0,1,0
2,2,"100,000 miles is way to long for one person to...","Jeff is a four letter name and 100,000 has six...","100,000 miles is longer than 100,000 km.",1,0,0
3,3,A human is a mammal,A human is omnivorous,A human has not stings,0,0,1
4,4,Giraffes can drink water from a lake.,A giraffe is not a human being.,.Giraffes usually eat leaves.,0,1,0


In [66]:
# creating three dataframe for more accuracy.
df1 = df[['OptionA','lblA']]
df2 = df[['OptionB','lblB']]
df3 = df[['OptionC','lblC']]

In [67]:
# same names for three cols for easy concate.
df1 = df1.rename(columns = {"OptionA": "sent", 
                                  "lblA":"label"})
df2 = df2.rename(columns = {"OptionB": "sent", 
                                  "lblB":"label"})
df3 = df3.rename(columns = {"OptionC": "sent", 
                                  "lblC":"label"})

In [68]:
# concate the three dataframes so we can get total 29991 entries.
df = pd.concat([df1, df2, df3], ignore_index=True,axis=0,sort=True)

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29991 entries, 0 to 29990
Data columns (total 2 columns):
label    29991 non-null int64
sent     29991 non-null object
dtypes: int64(1), object(1)
memory usage: 468.7+ KB


# Cleaning

    2.    Cleaning

        a.    Lower Case: Converting all 29,991 entries into lowercase.
        b.    Removing extra character: In this, the additional symbol like ?.'; is removed.
        c.    Stop words Removing: stop words like a the, and a terminated from the corpus.
        d.    Lemmatization: converting the text into the plural form. e.g., mice become a mouse.
        e.    Eliminating the repetitions: the word which occurs twice or more times it removes.

In [72]:
# lower case
df['sent'] = df['sent'].apply(lambda x: " ".join(x.lower() for x in x.split()))

# Removing extra symbol.
df['sent'] = df['sent'].str.replace('[^\w\s]',' ')

# removing stopwords like the, a, an
df['sent'] = df['sent'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df.head(1)

# Pural form through lemmatization eg. mice becomes mouse.
df['sent'] = df['sent'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df.head(1)

# correcting latter and repeatations.
def de_repeat(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1",text)
df['sent'] = df['sent'].apply(lambda x: " ".join(de_repeat(x) for x in x.split())) 

# 3. TRAIN TEST SPLIT

    3.    Train test split 

        a.    train-test-split: In train test split function, the dataset divided into two parts 1 with 70% of dataset and another with 30% of data. Further evaluated using three classification model.
        b.    For further evaluation, the dataset present in the test folder is taken and evaluated using three classification model.

In [73]:
from sklearn.model_selection import train_test_split

In [74]:
X = df['sent']
y = df['label']

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 4. Model Selection

    4.    Feature Extraction

        •    Machine Algorithms cannot take text data; instead, we need to perform feature extraction from text data for passing the text into a machine learning algorithm.   

        a.    Count Vectorization: It converts the raw text into DTM (Document term matrix) in this; each word occurrence counted.
        b.    TFIDF-Transformation: It transforms the count of count vectorization into frequency. Instead of 4.a and 4.b, we can use TFIDF-Vectorization for similar operations with one function.
        c.    TFIDF-Vectorization: It is just inverse of count vectorization; instead of filling DTM(Document term matrix) with word count, it calculates the inverse document frequency value for each word. Inverse document frequency is measured common or rare in a given corpus.
    
    4. Apply 3 Classification Model. 
        a.    Naive Bayes Classifier
            i.    Naive Bayes is a classification algorithm based on the principle of Bayes' Theorem.
            •    works with any numbers of class
            •    Simple to implement

        b.    K-nearest Neighbour
            i.    It is a classification algorithm that operates on basic principal.
            •    Simple to implement
            •    works with any numbers of class
            •    easy to add more data

        c.    Support Vector Machine (SVM)
            i.    It analysed data and recognizer patterns through classification or regression techniques.
            •    SVM can handle non-linear data using the bit trick.
            •    SVM can be used to solve both classification and regression problems.

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [78]:
# 4.1 Naive Bayes Classifier
text_nb = Pipeline([('tfidf',TfidfVectorizer(max_features=1000,analyzer='word',ngram_range=(1,2))),
                   ('clf',MultinomialNB())])

text_nb.fit(X_train,y_train)

prediction = text_nb.predict(X_test)

print(classification_report(y_test,prediction))

print("Confusion Matrix = \n",confusion_matrix(y_test,prediction))

print("accuracy = ",accuracy_score(y_test,prediction))

              precision    recall  f1-score   support

           0       0.69      0.95      0.80      5949
           1       0.64      0.18      0.28      3049

    accuracy                           0.69      8998
   macro avg       0.67      0.56      0.54      8998
weighted avg       0.68      0.69      0.63      8998

Confusion Matrix = 
 [[5637  312]
 [2495  554]]
accuracy =  0.6880417870637919


In [79]:
# 4.2 K-nearest Neighbor
text_knn = Pipeline([('tfidf',TfidfVectorizer(max_features=1000,analyzer='word',ngram_range=(1,3))),
                   ('clf',KNeighborsClassifier(n_neighbors=101))])

text_knn.fit(X_train,y_train)

prediction = text_knn.predict(X_test)

print(classification_report(y_test,prediction))

print("Confusion Matrix = \n",confusion_matrix(y_test,prediction))

print("accuracy = ",accuracy_score(y_test,prediction))

              precision    recall  f1-score   support

           0       0.66      1.00      0.80      5949
           1       0.61      0.01      0.02      3049

    accuracy                           0.66      8998
   macro avg       0.64      0.50      0.41      8998
weighted avg       0.64      0.66      0.53      8998

Confusion Matrix = 
 [[5931   18]
 [3021   28]]
accuracy =  0.6622582796176928


In [80]:
# 4.3 Support Vector Machine (SVM)
text_cf = Pipeline([('tfidf',TfidfVectorizer(max_features=1000,analyzer='word',ngram_range=(1,3))),
                   ('clf',SVC(gamma='auto'))])

text_cf.fit(X_train,y_train)

prediction = text_cf.predict(X_test)

print(classification_report(y_test,prediction))

print("Confusion Matrix = \n",confusion_matrix(y_test,prediction))

print("accuracy = ",accuracy_score(y_test,prediction))

              precision    recall  f1-score   support

           0       0.66      1.00      0.80      5949
           1       0.00      0.00      0.00      3049

    accuracy                           0.66      8998
   macro avg       0.33      0.50      0.40      8998
weighted avg       0.44      0.66      0.53      8998

Confusion Matrix = 
 [[5949    0]
 [3049    0]]
accuracy =  0.6611469215381196


/Users/ranjitsmac/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# END